In [1]:
%load_ext dotenv
%dotenv ../.env

In [1]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()])

In [ ]:
from strands import Agent
from strands.models import BedrockModel

# Multi Agent

## Agent Graphs: Building Multi-Agent Systems

In [12]:
import boto3
import os

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

In [8]:
import sys
import uuid

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
kb = KnowledgeBasesForAmazonBedrock()

In [17]:
knowledge_base_name = f'pr-agent-kb-{str(uuid.uuid4())[:5]}'
knowledge_base_description = "KB containing information about pristine PR articles"
s3_bucket_name = f"labs-bucket-{region}-{account_id}"
bucket_prefix = "data/kb/reflection/"

In [ ]:
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    s3_bucket_name,
    "amazon.titan-embed-text-v2:0",
    bucket_prefix
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

In [20]:
def upload_directory(path, bucket_name, bucket_prefix):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,f"{bucket_prefix}{file}")

In [ ]:
upload_directory("../good_prs", s3_bucket_name, bucket_prefix)

In [ ]:
# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

In [10]:
from strands import Agent, tool
from strands_tools import retrieve

In [16]:
@tool
def pr_writer_agent(query: str) -> str:
    """Public Relation content writer agent."""
    print("PR Writer agent...")
    
    pr_writer = Agent(
        model="us.amazon.nova-pro-v1:0",
        system_prompt=f"""You are to write a PR news for an upcoming show/movie based on the user request, and an optional recommended changes.

When writing the PR news, you should follow the following guidelines:

1. Headline: Capture Attention & Key Details - Be Concise & Informative: Use active verbs like "announces," "debuts," or "unveils." Include the title, release date/platform, and a hook (e.g., talent or franchise ties).
Example: "A24 Reveals Trailer for Horror Thriller ‘Nightfall’ Starring Florence Pugh, Premiering October 2024."
Highlight Exclusives: If applicable, note premieres at festivals (e.g., "Sundance 2025 Official Selection").

2. Lead Paragraph: The 5 Ws - Summarize Who (studio, talent), What (title, genre), When (release date), Where (platforms/theaters), and Why (unique angle, legacy, or cultural relevance).
Example: "Marvel Studios’ ‘Avengers: Legacy,’ directed by Ryan Coogler, will premiere in theaters globally on May 3, 2025, marking the MCU’s first reboot of the iconic franchise."

3. Key Content Elements - Synopsis: Offer a 1–2 paragraph teaser without spoilers. Emphasize uniqueness (e.g., "a dystopian love story set in 2140’s AI-dominated society").
Talent & Production Credentials: Highlight A-list actors, acclaimed directors, or award-winning crews. Mention prior successes (e.g., "From the Oscar-winning producer of Parasite").
Behind-the-Scenes (BTS) Insights: Share filming locations, technical innovations, or adaptations (e.g., "Shot in Iceland using cutting-edge VR cinematography").

4. Quotes: Add Authenticity - Source quotes from directors, leads, or producers that convey passion or vision.
Example: "‘This film redefines resilience,’ says director Jordan Peele. ‘We’re blending horror with social commentary in ways audiences haven’t seen.’"

5. Multimedia & Links - Embed or link to trailers, stills, and BTS content. Provide access to a password-protected media kit with HD assets.
Include official social handles and hashtags (e.g., #NightfallMovie).

6. Tailor for Target Audiences - Trade Outlets (Deadline, Variety): Focus on box office prospects, production budgets, or industry trends.
Lifestyle/Cultural Press (Vulture, Rolling Stone): Highlight themes, fashion, or soundtracks.
Local Media: Tie in filming locations or local talent.

7. SEO & Discoverability - Use keywords: Title, cast names, genre, and phrases like "upcoming sci-fi movies 2025."
Link to official sites or prior related content (e.g., a franchise’s timeline).

8. Logistics & Contact Info - Clearly state release platforms (e.g., "Streaming exclusively on Netstream starting November 12").
List premiere events or fan screenings.
Provide media contacts: Name, email, phone, and PR firm.

9. Professional Polish - Proofread: Ensure dates, names, and titles are error-free.
Tone: Match the project’s vibe (e.g., playful for comedies, sleek for action).
Style Guide: Adhere to outlet-specific rules (e.g., AP style).

10. Follow-Up & Amplification - Pitch Exclusives: Offer early access to trailers or interviews for top-tier outlets.
Post-Release: Monitor coverage, engage on social media, and update with new assets (e.g., poster drops).

You also have access to a knowledge base tool that contains pristine, high quality PRs that have been published in the past. You should use the knowledge base in assisting with the writing.
The knowledge base ID you have acess to is {kb_id} and region is {region}.
""",
        tools=[retrieve],
        callback_handler=None
    )
    return str(pr_writer(query))

In [ ]:
query = """Write a PR news for an upcoming movie given the following information: 
Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson 
"""

results = pr_writer_agent(query)

In [ ]:
print(results)

In [18]:
@tool
def pr_review_agent(query: str) -> str:
    """Public Relation content review agent."""
    print("PR Review agent...")
    
    pr_review_agent = Agent(
        system_prompt="""You are an AI assistant that performs a review for a given PR article and provide any feedback to improve the quality. 
As a PR article reviewer, you are given the specific guidelines to evaluate the quality of the document.

1. Write in clear, crisp sentences." - long or confusing sentences as "clunky" when they're difficult for readers to understand.

Here are the tips on to identify unclear sentences:

- Sentences longer than 40 words or spanning more than two lines
- Sentences that require multiple readings to understand
- Sentences with repeated words, especially "and" and "to"

2. Be specific. Don't leave anything open to interpretation or leave your reader guessing. 

Here are some clues your sentence isn't specific enough: 
-  it contains qualitative or subjective adjectives and adverbs (like "might," "often," or "may be"), 
- it leaves the reader asking questions.

Here are some ways to fix this problem: 
- clarify words or phrases that could be interpreted differently by different people, 
- use quantitative adjectives, like "3X faster" or "50% more time" where possible, and/or 

3. Write for a non-technical audience. Anyone should be able to understand your message without additional background information. Readers will often be left with questions if you've included concepts or undefined acronyms.

Here are some clues your sentence might be too technical: 
- your reader is left with questions, 
- it contains abbreviations or acronyms that aren't defined, and/or 
- it describes technology that an average person would not understand.

Here are some ways to fix this problem: 
3a. Include a short description of the show;
3b. define acronyms if used in the article

4. Proactively explain what makes a new show/movie interesting from similar existing show/movie. If there is any chance a reader might ask, "Isn't this movie similar to X?" then you need to address it. Avoid simply listing plots of the new movie.

Here are some ways to fix this problem:
- If there is an existing movie/show with similar plots as the new movie, address how they are different.

5. Connect the dots in a story.  You should make it clear enough how plots are related.

Here are some clues you're not connecting the dots: 
- your reader doesn't understand the plots you're writing about and/or 

Here is a way to fix this problem: 
Similar to unfolding the story, Have someone who hasn't heard of the story read the passage and ask them if it's clear to them and how everything is related.

6. Be consistent in names and descriptions. Don't confuse your reader by switching between different names or introducing additional characters near the end of your PR. Stick to one name and don't stray from the main plots.""",
callback_handler=None)
    return str(pr_review_agent(query))
    

In [ ]:
review_result = pr_review_agent(results)

In [ ]:
print(review_result)

In [21]:
pr_supervisor_agent = Agent(
        model="us.amazon.nova-premier-v1:0",
        system_prompt=f"""You are a supervisor AI agent. You are given 2 agents capable of following tasks:

1. PR Writer agent - Writes a draft of PR article based on the given instructions and/or recommendations.

2. PR Reviewer agent - Reviews the PR article generated by the PR Writer agent to provide recommendation to improve the PR article.

You are to leverage both the agents to work collaboratively and iteratively to arrive at the best version of the PR article. 

Here are the guidelines:
- You should only iterate the draft and review iterations for maximum of 3 time before returning the final version of the PR article to the user. 

The final draft of the PR must be a complete version formatted in markdown in the final response. Do not provide any explanation in the final response, return only the final PR article in markdown format.""",
        tools=[pr_writer_agent, pr_review_agent],
        callback_handler=None
    )

In [ ]:
query = """Write a PR news for an upcoming movie given the following information: 
Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson"""

pr_final_result = pr_supervisor_agent(query)

In [ ]:
print(str(pr_final_result))